In [1]:
# Step.1
import numpy as np
import pandas as pd
import copy

# Read the data
# excel name: data.xlsx
# Sheet Aij: 
# ij 0 1 2 3 4
# 0 1 1 0.95 0.9 0.85
# 1 N.A. 1 0.75 0.7 0.6
# 2 N.A. N.A. 1 0.9 0.85
# 3 N.A. N.A. N.A. 1 0.9
# 4 N.A. N.A. N.A. N.A. 1
# Sheet Demand: 
# Demand value
# D0 4600
# D1 800
# D2 600
# D3 460
# D4 1000
# Sheet Capacity: 
# Capacity Value
# B1 5200
# B2 4800
# B3 5000
# B4 10000

Aij = pd.read_excel('data.xlsx', sheet_name='Aij')
Aij = Aij.iloc[0:5,1:6].values
Aij = np.array(Aij)
Aij[Aij == 'N.A.'] = Aij[Aij == 'N.A.'] = np.nan

Demand = pd.read_excel('data.xlsx', sheet_name='Demand')
Demand = Demand.iloc[0:5,1:2].values

Capacity = pd.read_excel('data.xlsx', sheet_name='Capacity')
Capacity = Capacity.iloc[0:5,1:2].values

# Define the bounds for the variables
X_bounds = [(0, Demand[i, 0]) for i in range(4)]  # Assuming lower bound is 0 and upper bound is the corresponding demand
Y_bounds = [(0, 1) for _ in range(4)]  # For binary variables
K_bounds = [(0, 1) for _ in range(4)]  # For binary variables

# Combine all the bounds
all_bounds = X_bounds + Y_bounds + K_bounds

# Function to create an individual
def create_individual(bounds):
    individual = []
    for bound in bounds:
        if bound[1] - bound[0] == 1:  # Binary variables
            individual.append(np.random.choice([0, 1]))
        else:  # Continuous variables
            individual.append(np.random.uniform(low=bound[0], high=bound[1]))
    return individual



In [2]:
# Step.2
# Function to create a population
def create_population(bounds, size):
    return [create_individual(bounds) for _ in range(size)]



In [3]:
# Step.3
def calculate_fitness(individual, Aij, Demand, Capacity, M=1000000000):
    X = [float(x) for x in individual[:4]]
    Y = [int(round(y)) for y in individual[4:8]]
    K = [int(round(k)) for k in individual[8:]]

    X_min = 180
    X_max = 900
    D_0 = Demand[0, 0]
    B = Capacity

    penalty = 0
    for j in range(4):
        total_demand = D_0 * Aij[0, j+1] + sum([Aij[i, j+1] * X[i-1] for i in range(1, j + 2)])
        #1
        if total_demand > B[j, 0]:
            penalty += (total_demand - B[j, 0]) / (B[j, 0]) * M  # Could be dded epsilon to avoid division by zero ((B[j, 0] + 1e-10))
            #print(f"Constraint 1 violated for j={j}: total_demand={total_demand}, B[{j}]={B[j, 0]}")
        #2
        if X[j] > Demand[j+1, 0]:
            penalty += M
            #print(f"Constraint 2 violated for j={j}: X[{j}]={X[j]}, Demand[{j+1}]={Demand[j+1, 0]}")
        #3
        if X[j] < Demand[j+1, 0] * (1 - Y[j]):
            penalty += M
            #print(f"Constraint 3 violated for j={j}: X[{j}]={X[j]}, Demand[{j+1}] * (1 - Y[{j}])={Demand[j+1, 0] * (1 - Y[j])}")
        #4
        if X[j] < X_min * Y[j]:
            penalty += M
            #print(f"Constraint 4 violated for j={j}: X[{j}]={X[j]}, X_min * Y[{j}]={X_min * Y[j]}")
        #5
        if X[j] > Demand[j+1, 0] * (1 - Y[j]) + X_max * Y[j]:
            penalty += M
            #print(f"Constraint 5 violated for j={j}: X[{j}]={X[j]}, Demand[{j+1}] * (1 - Y[{j}]) + X_max * Y[{j}]={Demand[j+1, 0] * (1 - Y[j]) + X_max * Y[j]}")
        #6
        if Y[j] > M * K[j]:
            penalty += M
            #print(f"Constraint 6 violated for j={j}: Y[{j}]={Y[j]}, M * K[{j}]={M * K[j]}")
        #7
        if X[j] - Demand[j+1, 0] > M * (1 - K[j]):
            penalty += M
            #print(f"Constraint 7 violated for j={j}: X[{j}] - Demand[{j+1}]={X[j] - Demand[j+1, 0]}, M * (1 - K[{j}])={M * (1 - K[j])}")

    fitness = D_0 + sum(X) - penalty
    #print(f"Individual: {individual}, Penalty: {penalty}, Fitness: {fitness}")
    return fitness


#     except TypeError as te:
#         print("TypeError occurred")
#         print("Individual: ", individual)
#         print("Type of elements in Individual: ", [type(i) for i in individual])
#         print("X: ", X)
#         print("Type of elements in X: ", [type(i) for i in X])
#         print("Aij: ", Aij)
#         raise te

        

In [4]:
# Step.4
# You can continue using the roulette_wheel_selection or add tournament selection as an additional option.
def roulette_wheel_selection(population, fitnesses):
    # Calculate the total fitness of the population
    total_fitness = sum(fitnesses)

    # Generate a random number between 0 and the total fitness
    random_number = np.random.uniform(0, total_fitness)

    # Select an individual based on the random number
    cumulative_fitness = 0.0
    for i, individual in enumerate(population):
        cumulative_fitness += fitnesses[i]
        if cumulative_fitness > random_number:
            return individual

    # In case no individual is selected (should not happen if the fitnesses are correctly calculated)
    return None

# Example for tournament selection:
def tournament_selection(population, fitnesses, tournament_size):
    # Create a tournament
    tournament_indices = np.random.choice(len(population), tournament_size)
    tournament = [copy.deepcopy(population[i]) for i in tournament_indices]

    # Get the fitnesses of the tournament individuals
    tournament_fitnesses = [fitnesses[i] for i in tournament_indices]

    # Return the individual with the highest fitness
    return tournament[np.argmax(tournament_fitnesses)]



In [5]:
# Step.5
def single_point_crossover(parent1, parent2):
    # Choose a random index for the crossover point
    crossover_index = np.random.randint(1, len(parent1))

    # Create the offspring by combining the genes of the parents
    offspring1 = parent1[:crossover_index] + parent2[crossover_index:]
    offspring2 = parent2[:crossover_index] + parent1[crossover_index:]

    return offspring1, offspring2



In [6]:
# Step.6
def mutation(individual, mutation_rate, mutation_strength=0.1):
    individual_copy = individual.copy()  # create a copy to not modify the input
    # For each gene set in the individual
    for i in range(4):  # We're iterating over 4 groups of genes (Xj, yj, kj)
        # If a random number is less than the mutation rate
        if np.random.random() < mutation_rate:
            # Mutate the gene
            # Add a small random value for X genes
            individual_copy[i] += np.random.uniform(-mutation_strength * (Demand[i+1, 0] - individual_copy[i]), mutation_strength * (Demand[i+1, 0] - individual_copy[i]))
            # Ensure that the mutated gene respects its constraints
            individual_copy[i] = max(0, min(individual_copy[i], Demand[i+1, 0]))  # Adjusting the bounds to 0 and Dj
            
            # Flip the value for Y and K genes with probability mutation_rate
            if np.random.random() < mutation_rate:
                individual_copy[i+4] = 1 - individual_copy[i+4]  # Mutation for yj
                
            if np.random.random() < mutation_rate:
                individual_copy[i+8] = 1 - individual_copy[i+8]  # Mutation for kj
                
            # Now let's adjust Xj if it doesn't satisfy its constraints after yj or kj mutation
            Xmin = 180
            Xmax = 900
            if individual_copy[i] < (Demand[i+1, 0] * (1 - individual_copy[i+4])) or individual_copy[i] < (Xmin * individual_copy[i+4]):
                individual_copy[i] = Demand[i+1, 0] * (1 - individual_copy[i+4])
            if individual_copy[i] > (Demand[i+1, 0] * (1 - individual_copy[i+4]) + Xmax * individual_copy[i+4]):
                individual_copy[i] = Demand[i+1, 0] * (1 - individual_copy[i+4]) + Xmax * individual_copy[i+4]
    return individual_copy



In [7]:
# Step.7
def create_new_generation(population, Aij, Demand, Capacity, mutation_rate, M=10000000000):
    # Calculate the fitness of each individual in the population
    fitnesses = [calculate_fitness(individual, Aij, Demand, Capacity, M) for individual in population]

    # Create a new population
    new_population = []
    while len(new_population) < len(population):
        # Selection
        parent1 = tournament_selection(population, fitnesses, tournament_size=10)
        parent2 = tournament_selection(population, fitnesses, tournament_size=10)

        # Crossover
        offspring1, offspring2 = single_point_crossover(parent1, parent2)

        # Mutation
        mutation_strength = np.random.choice([0.1, 0.5], p=[0.9, 0.1])  # Occasionally choose a larger mutation strength
        offspring1 = mutation(offspring1, mutation_rate, mutation_strength)
        offspring2 = mutation(offspring2, mutation_rate, mutation_strength)

        # Add the new individuals to the new population
        new_population.extend([offspring1, offspring2])

    # If the new population size is greater than the original population size, truncate the new population
    if len(new_population) > len(population):
        new_population = new_population[:len(population)]

    return new_population
#     except TypeError as te:
#         print("TypeError occurred in new generation creation")
#         print("Population: ", population)
#         print("Type of elements in Population: ", [type(i) for i in population])
#         raise te
     
    

In [8]:
# Step.8
def genetic_algorithm(Aij, Demand, Capacity, mutation_rate, num_generations, population_size):
    # Create a population
    population = create_population(all_bounds, population_size)

    # Repeat the process for a certain number of generations
    for _ in range(num_generations):
        # Create a new generation
        population = create_new_generation(population, Aij, Demand, Capacity, mutation_rate)

    # After the last generation, return the best solution
    fitnesses = [calculate_fitness(individual, Aij, Demand, Capacity, M=1000000000) for individual in population]
    best_solution = population[np.argmax(fitnesses)]
    best_fitness = max(fitnesses)

    return best_solution, best_fitness



In [9]:
# Run
import time

start_time = time.time()

best_solution, best_fitness = genetic_algorithm(Aij, Demand, Capacity, mutation_rate=0.1, num_generations=1000, population_size=100)

end_time = time.time()

runtime = end_time - start_time

# Print the best solution, its fitness, and the runtime
print("Best solution: ", best_solution)
print("Fitness of the best solution: ", best_fitness)
print("Runtime(min): ", runtime/60)



Best solution:  [328.45552640076824, 183.64473673758366, 460.0, 1000.0, 1, 1, 1, 0, 1, 1, 1, 0]
Fitness of the best solution:  6572.100263138352
Runtime(min):  0.8148149291674296


In [10]:
#Groubi
gurobi_solution = [333.333, 180, 460, 1000, 1, 1, 0, 0, 1, 1, 0, 0]
optimal_fitness = calculate_fitness(gurobi_solution, Aij, Demand, Capacity)

print(f"Fitness value of the Gurobi solution: {optimal_fitness}")



Fitness value of the Gurobi solution: 6573.3330000000005


In [11]:
# Error
ga_solution = best_solution
ga_fitness = best_fitness
optimal_solution = gurobi_solution
optimal_fitness = 6573.3330000000005

# calculate the relative error in fitness
relative_error_fitness = abs(ga_fitness - optimal_fitness) / optimal_fitness * 100

# Filter out binary values
ga_solution_continuous = [x for i, x in enumerate(ga_solution) if i < 4]
optimal_solution_continuous = [x for i, x in enumerate(optimal_solution) if i < 4]

# Calculate the mean absolute percentage error in solution values (excluding binary variables)
mape_solution_values = sum(abs(ga - opt) / (abs(opt) + 1e-7) for ga, opt in zip(ga_solution_continuous, optimal_solution_continuous)) / len(ga_solution_continuous) * 100

print('Relative Error in Fitness (%):', relative_error_fitness)
print('Mean Absolute Percentage Error in Solution Values (%):', mape_solution_values)



Relative Error in Fitness (%): 0.018753604322933975
Mean Absolute Percentage Error in Solution Values (%): 0.8720243211378048
